# Module 1b: Agent with Skills and Context

In this module, you'll run the **enhanced Claude agent WITH skills and CLAUDE.md context** and compare it to the basic agent from Module 1a.

## Learning Objectives

By the end of this module, you will:
- Enable skills discovery via `setting_sources=["project"]`
- Load the `Skill` tool for domain expertise
- Use CLAUDE.md for project context
- See dramatic improvement in agent accuracy

## What to Expect

The enhanced agent will:
- Load appropriate skill FIRST based on query domain
- Know exact column names and data types
- Use pre-calculated metrics correctly
- Complete tasks in fewer turns with higher accuracy

## Step 1: Load Environment

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Set up paths
workshop_root = Path("..").resolve()
sys.path.insert(0, str(workshop_root))

# Load environment
load_dotenv(workshop_root / ".env")

# CRITICAL: Ensure Claude Agent SDK uses Bedrock (not Claude API key)
os.environ["CLAUDE_CODE_USE_BEDROCK"] = "1"

print(f"Workshop root: {workshop_root}")
print(f"AWS Region: {os.getenv('AWS_REGION')}")
print(f"Using Bedrock: {os.getenv('CLAUDE_CODE_USE_BEDROCK')}")

Workshop root: /Users/mmelli/Documents/projects/claude-code-agentcore/agentic-ai-with-claude-agent-sdk-and-amazon-bedrock-agentcore
AWS Region: us-west-2
Using Bedrock: 1


## Step 2: Explore the Skills

Let's look at what skills are available and what they provide.

In [2]:
# List available skills
skills_dir = workshop_root / ".claude" / "skills"

print("Available Skills:")
print("=" * 50)

for skill_folder in skills_dir.iterdir():
    if skill_folder.is_dir():
        skill_file = skill_folder / "SKILL.md"
        if skill_file.exists():
            with open(skill_file, 'r') as f:
                content = f.read()
            
            # Extract name and description from frontmatter
            import re
            name_match = re.search(r'name:\s*(.+)', content)
            desc_match = re.search(r'description:\s*(.+)', content)
            
            name = name_match.group(1) if name_match else skill_folder.name
            desc = desc_match.group(1)[:100] if desc_match else "No description"
            
            print(f"\n{name}")
            print(f"  Path: {skill_file.relative_to(workshop_root)}")
            print(f"  Description: {desc}...")

Available Skills:

financial-analytics
  Path: .claude/skills/financial/SKILL.md
  Description: Analyze tuition payments and revenue, outstanding balances and payment status, scholarship awards an...

enrollment-analytics
  Path: .claude/skills/enrollment/SKILL.md
  Description: Analyze student enrollment trends, course capacity utilization, enrollment patterns by semester/depa...

academic-performance
  Path: .claude/skills/academic/SKILL.md
  Description: Analyze student GPA and academic performance, grade distributions, honor roll and dean's list studen...


In [3]:
# Let's look at the content of one skill
enrollment_skill = skills_dir / "enrollment" / "SKILL.md"

with open(enrollment_skill, 'r') as f:
    content = f.read()

print("Enrollment Analytics Skill (first 2000 chars):")
print("=" * 50)
print(content[:2000])
print("\n... [truncated]")

Enrollment Analytics Skill (first 2000 chars):
---
name: enrollment-analytics
description: Analyze student enrollment trends, course capacity utilization, enrollment patterns by semester/department/major, student retention, course popularity, and over-enrolled or under-enrolled courses. Use when user asks about enrollment, course registration, capacity planning, or enrollment status tracking.
---

# Enrollment Analytics

## Primary Table
**student_enrollment_analytics** - Metadata: `data/metadata/student_enrollment_analytics.yaml`

## Related Tables
- **student_activity_engagement** - Extracurricular involvement
- **library_usage_patterns** - Library utilization
- **department_summary_metrics** - Department-level enrollment

## Pre-Calculated Metrics
- Utilization: `utilization_rate_pct`, `current_enrollment_count`, `course_max_enrollment`
- Flags: `is_current_semester`, `days_since_enrollment`

## Key Thresholds
**Utilization Rate:**
- >= 100%: Over-enrolled (need more sections)
- 90-

## Step 3: Explore CLAUDE.md

The `CLAUDE.md` file provides overall project context that the agent can reference.

In [4]:
# Look at CLAUDE.md structure
claude_md = workshop_root / "CLAUDE.md"

with open(claude_md, 'r') as f:
    content = f.read()

print(f"CLAUDE.md: {len(content):,} characters")
print("\nFirst 1500 characters:")
print("=" * 50)
print(content[:1500])

CLAUDE.md: 4,802 characters

First 1500 characters:
# Student Analytics Agent - Project Context

## Overview
Student Analytics AI Agent built on Claude Agent SDK. Queries Amazon Athena database for student management data analysis through natural language.

## Critical Rule
ALWAYS load appropriate skills and read BOTH the metadata file AND sample data file FIRST before writing any SQL query.

**For each table, review:**
- `data/metadata/<table_name>.yaml` - Complete column definitions, data types, and possible values
- `data/metadata/<table_name>_sample_data.csv` - Sample rows showing actual data format and examples

DO NOT guess or assume column names or values.

## Project Structure
```
/
├── results/
│   ├── raw/{request_id}/      # Athena query results (CSV)
│   └── processed/{request_id}/ # Processed data and visualizations
├── .claude/
│   └── skills/           # Domain-specific agent skills
│       ├── academic/SKILL.md
│       ├── enrollment/SKILL.md
│       └── financial/SKILL

## Step 4: Enhanced Agent Configuration

The key differences from the basic agent:

```python
# Enhanced agent configuration
options = ClaudeAgentOptions(
    system_prompt=full_context_prompt,
    allowed_tools=["Skill", "Read", "Write", "Bash"],  # SKILL tool enabled!
    setting_sources=["project"],  # Auto-discover skills from .claude/skills/
    cwd=project_root,
    max_turns=30
)
```

In [ ]:
from claude_agent_sdk import ClaudeAgentOptions, tool, create_sdk_mcp_server, ClaudeSDKClient
from tools.athena_tools import AthenaQueryExecutor

# Get configuration
athena_database = os.getenv("ATHENA_DATABASE", "student_analytics")
athena_output = os.getenv("ATHENA_OUTPUT_LOCATION")
aws_region = os.getenv("AWS_REGION", "us-east-1")

# Initialize Athena executor
athena_executor = AthenaQueryExecutor(
    database=athena_database,
    output_location=athena_output,
    results_dir='./results/raw',
    region=aws_region
)

# Create the Athena MCP tool
@tool("execute_athena_query", "Execute SQL queries against Amazon Athena database and download results", {
    "query": str,
    "local_filename": str
})
async def execute_athena_query(args):
    """Execute SQL query on Athena and download results."""
    try:
        query_text = args.get("query", "")
        local_filename = args.get("local_filename", "query_results.csv")
        
        result = athena_executor.execute_and_download(
            query=query_text,
            local_filename=local_filename
        )
        
        response_text = f"""Query completed successfully!
Data scanned: {result.get('data_scanned_bytes', 0) / (1024**2):.2f} MB
Execution time: {result.get('execution_time_ms', 0) / 1000:.2f} seconds
Results downloaded to: {result['local_file']}"""
        
        return {"content": [{"type": "text", "text": response_text}]}
    except Exception as e:
        return {"content": [{"type": "text", "text": f"Error executing query: {str(e)}"}], "isError": True}

# Create MCP server with the Athena tool
athena_server = create_sdk_mcp_server(
    name="athena",
    version="1.0.0",
    tools=[execute_athena_query]
)

# Load CLAUDE.md content
with open(workshop_root / "CLAUDE.md", 'r') as f:
    claude_md_content = f.read()

# Full system prompt with project context
enhanced_system_prompt = f"""You are a Student Analytics AI Agent.

{claude_md_content}

Use the execute_athena_query tool to run SQL queries against the database.
"""

print("Enhanced agent configured with:")
print("  - CLAUDE.md context")
print("  - Skill tool enabled")
print("  - MCP Athena tool: mcp__athena__execute_athena_query")
print("  - Project setting sources")
print(f"  - Total context: {len(enhanced_system_prompt):,} chars")

Enhanced agent configured with:
  - CLAUDE.md context
  - Skill tool enabled
  - MCP Athena tool: mcp__athena__execute_athena_query
  - Project setting sources
  - Total context: 4,918 chars


## Step 5: Run the Enhanced Agent

Let's run the SAME queries from Module 1a and compare the results.

In [6]:
import anyio

async def run_enhanced_query(user_query: str):
    """Run a query using the enhanced agent (with skills)"""
    
    # Configure with skills enabled AND MCP Athena tool
    options = ClaudeAgentOptions(
        system_prompt=enhanced_system_prompt,
        allowed_tools=["Skill", "Read", "Write", "Bash", "mcp__athena__execute_athena_query"],  # Skill tool!
        mcp_servers={"athena": athena_server},
        setting_sources=["project"],  # Auto-discover skills
        cwd=str(workshop_root),
        max_turns=30
    )
    
    print("=" * 70)
    print("ENHANCED AGENT (With Skills & CLAUDE.md)")
    print("=" * 70)
    print(f"\nQuery: {user_query}\n")
    print("-" * 70)
    
    tool_calls = []
    skills_loaded = []
    
    async with ClaudeSDKClient(options=options) as client:
        await client.query(user_query)
        
        async for message in client.receive_response():
            if hasattr(message, 'subtype'):
                if message.subtype == 'success':
                    duration = getattr(message, 'duration_ms', 0) / 1000
                    cost = getattr(message, 'total_cost_usd', 0)
                    turns = getattr(message, 'num_turns', 0)
                    print(f"\n" + "=" * 70)
                    print(f"Completed: {duration:.1f}s | ${cost:.4f} | {turns} turns")
                    print(f"Skills loaded: {skills_loaded}")
                    print(f"Total tool calls: {len(tool_calls)}")
                    print("=" * 70)
                    continue
            
            if hasattr(message, 'content'):
                for content in (message.content if isinstance(message.content, list) else [message.content]):
                    if hasattr(content, 'text'):
                        text = content.text
                        # Truncate long skill content
                        if text.startswith("Base directory"):
                            print(text.split('\n')[0])
                        elif len(text) > 500 and "Query Patterns" in text:
                            print("[Skill content loaded - showing summary...]")
                        else:
                            print(text)
                    elif hasattr(content, 'name'):
                        tool_calls.append(content.name)
                        if content.name == "Skill":
                            skill_name = content.input.get('skill', 'unknown')
                            skills_loaded.append(skill_name)
                            print(f"\n[Loading skill: {skill_name}]")
                        elif content.name == "mcp__athena__execute_athena_query":
                            query_sql = content.input.get('query', '')
                            filename = content.input.get('local_filename', '')
                            print(f"\n[Executing SQL:]")
                            print("-" * 40)
                            print(query_sql[:300])
                            print("-" * 40)
                            print(f"Output: {filename}")
                        elif content.name == "Read":
                            file_path = content.input.get('file_path', '')
                            if 'metadata' in file_path:
                                print(f"\n[Reading metadata: {file_path.split('/')[-1]}]")

### Query 1: "How many students are currently enrolled?"

Watch how the enhanced agent:
1. Loads the `enrollment-analytics` skill
2. Reads the correct metadata file
3. Uses exact column names in the query

In [7]:
await run_enhanced_query("How many students are currently enrolled?")

ENHANCED AGENT (With Skills & CLAUDE.md)

Query: How many students are currently enrolled?

----------------------------------------------------------------------
I'll help you find the number of currently enrolled students. Let me start by loading the enrollment analytics skill and reviewing the relevant table metadata.

[Loading skill: enrollment]

[Reading metadata: student_enrollment_analytics.yaml]

[Reading metadata: student_enrollment_analytics_sample_data.csv]
Base directory for this skill: /Users/mmelli/Documents/projects/claude-code-agentcore/agentic-ai-with-claude-agent-sdk-and-amazon-bedrock-agentcore/.claude/skills/enrollment
Now I'll execute a query to find the number of currently enrolled students. Based on the metadata and skill instructions, I need to use all three filters for "currently enrolled" students.

[Executing SQL:]
----------------------------------------
SELECT COUNT(DISTINCT student_id) as total_enrolled_students
FROM student_enrollment_analytics
WHERE enro

### Query 2: "Show me the top 5 students with the highest GPA on the honor roll"

Notice the agent uses the `is_honor_roll` pre-calculated flag!

In [8]:
await run_enhanced_query("Show me the top 5 students with the highest GPA who are on the honor roll")

ENHANCED AGENT (With Skills & CLAUDE.md)

Query: Show me the top 5 students with the highest GPA who are on the honor roll

----------------------------------------------------------------------
I'll help you find the top 5 students with the highest GPA who are on the honor roll. Let me start by loading the academic performance skill and reviewing the relevant table metadata.

[Loading skill: academic]

[Reading metadata: student_academic_performance.yaml]

[Reading metadata: student_academic_performance_sample_data.csv]
Base directory for this skill: /Users/mmelli/Documents/projects/claude-code-agentcore/agentic-ai-with-claude-agent-sdk-and-amazon-bedrock-agentcore/.claude/skills/academic
Perfect! Now I have the academic skill loaded and the table metadata. I can see that:
- The `is_honor_roll` flag indicates students with GPA >= 3.5
- I need to use `student_overall_gpa` for the GPA value
- I should GROUP BY student fields to avoid duplicates from multiple course records

Let me now e

### Query 3: "What's the scholarship coverage rate by major?"

The agent uses the pre-calculated `scholarship_coverage_rate_pct` field!

In [ ]:
await run_enhanced_query("What's the scholarship coverage rate by major?")

## Step 6: Comparison Summary

| Aspect | Basic Agent | Enhanced Agent |
|--------|-------------|----------------|
| Column names | Guesses, often wrong | Reads from metadata |
| Pre-calculated metrics | Doesn't know about them | Uses `is_honor_roll`, `scholarship_coverage_rate_pct`, etc. |
| Query patterns | Trial and error | Follows skill examples |
| Turns required | More (due to errors) | Fewer (correct first time) |
| Cost | Higher | Lower |
| Reliability | Inconsistent | Consistent |

## Step 7: Try Your Own Queries

Test the enhanced agent with your own questions!

In [ ]:
# Try your own query!
# Examples:
# - "Which courses have the highest failure rate?"
# - "What's the average tuition revenue by department?"
# - "Show me students at academic risk"

your_query = "Which departments have the highest average GPA?"
await run_enhanced_query(your_query)

## Key Takeaways

### What Skills Provide

1. **Domain Expertise** - Query patterns, best practices, common pitfalls
2. **Table Knowledge** - Which tables to use for each question type
3. **Column Names** - Exact field names from metadata
4. **Pre-calculated Metrics** - Use flags like `is_honor_roll` instead of complex WHERE clauses

### What CLAUDE.md Provides

1. **Project Context** - Overall understanding of the codebase
2. **Workflow Guidelines** - Step-by-step process for handling queries
3. **Tool Usage** - How to use AthenaTools correctly
4. **Best Practices** - File naming, error handling, etc.

### Context Engineering Impact

The combination of skills + CLAUDE.md is a form of **context engineering** - carefully designing the information available to the agent to maximize its effectiveness.

## Next Steps

Continue to:

**[Module 2: Deploy to AgentCore](../module-2-agentcore-deployment/2-deploy-to-agentcore.ipynb)**

In the next module, you'll:
- Configure your agent for Amazon Bedrock AgentCore
- Build and test locally with containers
- Deploy to production
- Invoke your deployed agent via API

---

*Workshop: Build Agentic AI Applications with Claude Agent SDK and Amazon Bedrock AgentCore*